In [ ]:
import openmc
import math
import os
import numpy as np
import matplotlib.pyplot as plt

def KandGodiva():
    U235 = openmc.Material(1, "U235", temperature=1200)
    # Add nuclides to uo2
    U235.add_nuclide('U235', 1)

    r_godiva=0.1016
    U235.volume = 4*math.pi*r_godiva**2/3


    mats = openmc.Materials([U235])
    mats.export_to_xml()

    fuel_or = openmc.Sphere(r=r_godiva)

    fuel_region = -fuel_or

    fuel = openmc.Cell(1, 'fuel')
    fuel.fill = U235
    fuel.region = fuel_region

    pitch = 500
    #we define the x and y planes with boundary condition
    left = openmc.XPlane(x0=-pitch/2, boundary_type='vacuum')
    right = openmc.XPlane(x0=pitch/2, boundary_type='vacuum')
    bottom = openmc.YPlane(y0=-pitch/2, boundary_type='vacuum')
    top = openmc.YPlane(y0=pitch/2, boundary_type='vacuum')
    z1 = openmc.ZPlane(z0=pitch/2, boundary_type='vacuum')
    z2 = openmc.ZPlane(z0=-pitch/2, boundary_type='vacuum')

    #outside of left and inside of right, outside of bottom, and inside of top and outside of fuel outer cylinder
    outer_region = +fuel_or

    outer = openmc.Cell(name='outer')
    outer.region = outer_region

    root = openmc.Universe(cells=(fuel, outer))

    geom = openmc.Geometry()
    geom.root_universe = root
    geom.export_to_xml()

    point = openmc.stats.Point((0, 0, 0))
    src = openmc.Source(space=point)

    settings = openmc.Settings()
    settings.source = src
    settings.batches = 100
    settings.inactive = 10
    settings.particles = 500
    settings.output = {'tallies': False}
    settings.export_to_xml()

    model = openmc.model.Model(geom, mats, settings)
    
    return model

output=KandGodiva().run()
keff=openmc.StatePoint(output).k_combined.nominal_value
print(keff)